# Data Preprocessing - work on text cutting + sentiment mapping. 

### Import relevant libraries

In [1]:
import pandas as pd
import datetime
import re
import heapq
import numpy as np
import nltk


In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\92335\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:

pip install bert-extractive-summarizer

You should consider upgrading via the 'c:\users\92335\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


SyntaxError: invalid syntax (<ipython-input-142-41df7fc4c59c>, line 1)

### Import and build news data

In [2]:
news_data = pd.read_csv('SBP_Master_Data.csv', names = ['Date', 'News'], index_col = 0) #Original raw data file

In [3]:
news_data['Polarity'] = [0 for i in range(len(news_data))] #Initialize polarities to zero

In [4]:
news_data['Sentiment'] = [0 for i in range(len(news_data))] #Initialize polarities to zero

In [5]:
news_data.drop_duplicates(subset ="News", 
                     keep = False, inplace = True) 
  

In [6]:
news_data.head()

,Date,News,Polarity,Sentiment
NaN,Date,SBP Press Release,0,0
0.0,2006-01-01 00:00:00+00:00,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,0
1.0,2006-06-16 00:00:00+00:00,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0
2.0,2006-03-17 00:00:00+00:00,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0
3.0,2006-05-26 00:00:00+00:00,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0


In [7]:
news_data.info()

news_data = news_data.iloc[1:]
news_data.head()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1347 entries, nan to 1347.0
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       1347 non-null   object
 1   News       1347 non-null   object
 2   Polarity   1347 non-null   int64 
 3   Sentiment  1347 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 52.6+ KB


,Date,News,Polarity,Sentiment
0.0,2006-01-01 00:00:00+00:00,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,0
1.0,2006-06-16 00:00:00+00:00,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0
2.0,2006-03-17 00:00:00+00:00,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0
3.0,2006-05-26 00:00:00+00:00,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0
4.0,2006-06-09 00:00:00+00:00,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,0


In [8]:
news_data['Date'] = pd.to_datetime(news_data['Date']) #Convert to datetime
news_data['Date'] = news_data["Date"].dt.strftime("%d %m %Y") #Format date
news_data['News'].map(str) #Convert news to string

0.0       September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...
1.0       16"" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...
2.0       16'" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...
3.0       26" May, 2006\r\n\r\nState Bank to issue Rs.50...
4.0       9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...
                                ...                        
1343.0     \r\n\r\nExternal Relations Department\r\n\r\n...
1344.0     \r\n\r\nExternal Relations Department\r\n\r\n...
1345.0     \r\n\r\nExternal Relations Department\r\n\r\n...
1346.0    MONETARY POLICY COMMITTEE\r\nSTATE BANK OF PAK...
1347.0     \r\n\r\naK External Relations Department\r\n\...
Name: News, Length: 1346, dtype: object

In [9]:
news_data.head()

,Date,News,Polarity,Sentiment
0.0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,0
1.0,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0
2.0,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0
3.0,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0
4.0,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,0


In [10]:
news_data.reset_index(drop = True, inplace = True)

In [11]:
news_data.head()

,Date,News,Polarity,Sentiment
0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,0
1,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0
2,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0
3,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0
4,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,0


In [12]:
len(news_data['News'][0])

4056

In [13]:
news_data["Cut"] = ""

In [14]:
news_data.head()

,Date,News,Polarity,Sentiment,Cut
0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,0,
1,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0,
2,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0,
3,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0,
4,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,0,


In [18]:
# Removing Square Brackets and Extra Spaces
stopwords = nltk.corpus.stopwords.words('english')
newss = []
for article_text in news_data['News']: 
    #print(article_text)
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
    article_text = re.sub(r'\s+', ' ', article_text)
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)
    sentence_list = nltk.sent_tokenize(article_text)
    word_frequencies = {}
    for word in nltk.word_tokenize(formatted_article_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
        maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
    
    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)
    newss.append(summary)

news_data['Cut'] = np.resize(newss,len(news_data))

In [19]:
news_data.head()


,Date,News,Polarity,Sentiment,Cut
0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,0,"However, she said, policies should be made to ..."
1,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0,"$108.71 million, $61.32 million, $71.24 millio..."
2,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0,"$118.71 million, $60.16 million, $46.10 millio..."
3,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0,Issuance of larger denominated note also econo...
4,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,0,The flexibility in detemm ining the income thr...


In [20]:
len(news_data['Cut'][0])

1082

In [43]:
news_data["News Length"]= news_data["News"].str.len() 

In [44]:
news_data["Cut Length"]= news_data["Cut"].str.len() 

In [45]:
news_data.head()

,Date,News,Polarity,Sentiment,Cut,neg,neu,pos,compound,neu-compound,News Length,Cut Length
0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,1,"However, she said, policies should be made to ...",0.085,0.841,0.074,-0.2337,1.0747,4056,1082
1,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0,"$108.71 million, $61.32 million, $71.24 millio...",0.000,0.870,0.130,0.8126,0.0574,3063,450
2,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0,"$118.71 million, $60.16 million, $46.10 millio...",0.000,0.878,0.122,0.8658,0.0122,2653,602
3,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0,Issuance of larger denominated note also econo...,0.014,0.896,0.091,0.8885,0.0075,2425,982
4,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,1,The flexibility in detemm ining the income thr...,0.090,0.854,0.057,-0.5948,1.4488,2568,490


In [46]:
news_data.describe()

,Polarity,Sentiment,neg,neu,pos,compound,neu-compound,News Length,Cut Length
count,1346.0,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000
mean,0.0,0.142645,0.022783,0.830212,0.097238,0.484605,0.345607,2139.934621,612.303120
std,0.0,0.936104,0.035398,0.215221,0.098453,0.478284,0.516145,971.827335,345.575153
min,0.0,-1.000000,0.000000,0.000000,0.000000,-0.959800,-0.926400,4.000000,0.000000
25%,0.0,-1.000000,0.000000,0.816000,0.033000,0.000000,-0.062200,1320.500000,321.000000
50%,0.0,1.000000,0.000000,0.879000,0.084000,0.648600,0.154450,2279.000000,642.000000
75%,0.0,1.000000,0.037000,0.935000,0.138750,0.911575,0.712075,2860.500000,918.000000
max,0.0,1.000000,0.285000,1.000000,1.000000,0.994100,1.787100,4419.000000,1282.000000


### Import and stock prices as daily difference

In [54]:
org = pd.read_csv('ABOT PA Equity.csv')

In [50]:
diff = pd.read_csv('Companies diff/ABOT PA Equity.csv') #Original raw data file

In [51]:
diff.describe()

,Open,High,Low,Close,Volume
count,5157.000000,5157.000000,5157.000000,5157.000000,5157.000000
mean,0.147057,0.149578,0.145137,0.147993,5.805895
std,8.392596,6.892363,7.040883,6.328886,62090.918356
min,-88.884400,-58.971300,-41.226500,-45.014800,-925733.000000
25%,-0.763300,-0.825200,-0.660600,-0.731900,-3408.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.854600,0.882200,0.811400,0.757100,3200.000000
max,83.756400,43.741300,52.988800,52.143600,918700.000000


In [23]:
diff['Date'] = pd.to_datetime(diff['Date']) #Convert date to datetime

In [56]:
org = org.iloc[1:]
org.head()

,Date,Open,High,Low,Close,Volume
1,1/3/2001,7.6285,7.6285,7.6285,7.6285,2509
2,1/4/2001,7.6285,7.6285,7.6285,7.6285,2509
3,1/5/2001,7.6285,7.6285,7.6285,7.6285,2509
4,1/8/2001,7.6285,7.6285,7.6285,7.6285,2509
5,1/9/2001,7.6285,7.6285,7.6285,7.6285,2509


In [52]:
diff.head()

,Open,High,Low,Close,Volume,Date
0,0.0,0.0,0.0,0.0,0.0,2001-01-03
1,0.0,0.0,0.0,0.0,0.0,2001-01-04
2,0.0,0.0,0.0,0.0,0.0,2001-01-05
3,0.0,0.0,0.0,0.0,0.0,2001-01-08
4,0.0,0.0,0.0,0.0,0.0,2001-01-09


In [57]:
diff.tail()

,Open,High,Low,Close,Volume,Date
5152,0.99,-1.0,0.00,-11.20,250.0,2020-10-02
5153,-6.00,-1.0,-33.00,-3.58,25250.0,2020-10-05
5154,-10.00,-9.0,23.00,-2.12,-22950.0,2020-10-06
5155,19.55,11.0,0.05,13.62,12700.0,2020-10-07
5156,6.45,9.0,16.05,6.78,6600.0,2020-10-08


In [58]:
diff.describe()

,Open,High,Low,Close,Volume
count,5157.000000,5157.000000,5157.000000,5157.000000,5157.000000
mean,0.147057,0.149578,0.145137,0.147993,5.805895
std,8.392596,6.892363,7.040883,6.328886,62090.918356
min,-88.884400,-58.971300,-41.226500,-45.014800,-925733.000000
25%,-0.763300,-0.825200,-0.660600,-0.731900,-3408.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.854600,0.882200,0.811400,0.757100,3200.000000
max,83.756400,43.741300,52.988800,52.143600,918700.000000


### Make relevant functions to map polarities and take into account weekends

In [59]:
def pol(val, mean, std):
    #print(val)
    if val >= mean + std:
        
        return 1
    
    elif val < mean-std:
        
        return -1
    
    else:
        
        return 0

# Year, month, day	
def check(x):
    d3 = datetime.datetime.strptime(x, '%d %m %Y')
    d2 = datetime.datetime.strptime(x, '%d %m %Y').weekday()
   # print(d3)
    ##print(d2)

    if (d2 == 5):
        d1 = d3 - datetime.timedelta(days = 1)
    elif (d2 == 6):
        d1 = d3 - datetime.timedelta(days = 2)
    else:
        d1 = d3
    #print(d1)    
    return d1


check('08 02 2009')

datetime.datetime(2009, 2, 6, 0, 0)

In [60]:
news_data.head()


,Date,News,Polarity,Sentiment,Cut,neg,neu,pos,compound,neu-compound,News Length,Cut Length
0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,1,"However, she said, policies should be made to ...",0.085,0.841,0.074,-0.2337,1.0747,4056,1082
1,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0,"$108.71 million, $61.32 million, $71.24 millio...",0.000,0.870,0.130,0.8126,0.0574,3063,450
2,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0,"$118.71 million, $60.16 million, $46.10 millio...",0.000,0.878,0.122,0.8658,0.0122,2653,602
3,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0,Issuance of larger denominated note also econo...,0.014,0.896,0.091,0.8885,0.0075,2425,982
4,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,1,The flexibility in detemm ining the income thr...,0.090,0.854,0.057,-0.5948,1.4488,2568,490


In [29]:
x = len(news_data)
print(x)

1346


In [61]:
print(len(diff))


5157


In [62]:
len(news_data)

1346

In [63]:
news_data.head(10)

,Date,News,Polarity,Sentiment,Cut,neg,neu,pos,compound,neu-compound,News Length,Cut Length
0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,1,"However, she said, policies should be made to ...",0.085,0.841,0.074,-0.2337,1.0747,4056,1082
1,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0,"$108.71 million, $61.32 million, $71.24 millio...",0.000,0.870,0.130,0.8126,0.0574,3063,450
2,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0,"$118.71 million, $60.16 million, $46.10 millio...",0.000,0.878,0.122,0.8658,0.0122,2653,602
3,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0,Issuance of larger denominated note also econo...,0.014,0.896,0.091,0.8885,0.0075,2425,982
4,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,1,The flexibility in detemm ining the income thr...,0.090,0.854,0.057,-0.5948,1.4488,2568,490
5,03 07 2006,"1"" July, 2006\r\n\r\nSBP GOVERNOR MEETS APTMA\...",0,1,"Sympathizing with the textile exporters, Gover...",0.074,0.843,0.083,0.3384,0.5046,3167,991
6,27 01 2006,"26"" January, 2006\r\n\r\nSTATE BANK ISSUES ITS...",0,1,The distinguishing feature of the Monetary Pol...,0.000,1.000,0.000,0.0000,1.0000,606,281
7,09 06 2006,"9"" June, 2006\r\n\r\nSBP IMPOSES 50% CASH MARG...",0,1,The State Bank has also directed the banks to ...,0.000,0.960,0.040,0.3400,0.6200,1125,355
8,18 01 2006,"18"" January, 2006\r\n\r\nPAKISTAN RECEIVES OVE...",0,1,"$99.06 million, $57.66 million, $49.86 million...",0.000,0.920,0.080,0.7430,0.1770,2607,597
9,28 06 2006,"27% June, 2006\r\n\r\n \r\n\r\nSTATE BAN K ISS...",0,1,For detailed Guidelines and Prudential Regulat...,0.000,0.839,0.161,0.3182,0.5208,2602,93


### Label data with sentiment

In [64]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\92335\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [65]:
vader = SentimentIntensityAnalyzer()
scores = news_data['Cut'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
news_data = news_data.join(scores_df, rsuffix='_right')

In [66]:
news_data.head(25)

,Date,News,Polarity,Sentiment,Cut,neg,neu,pos,compound,neu-compound,News Length,Cut Length,neg_right,neu_right,pos_right,compound_right
0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,1,"However, she said, policies should be made to ...",0.085,0.841,0.074,-0.2337,1.0747,4056,1082,0.085,0.841,0.074,-0.2337
1,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0,"$108.71 million, $61.32 million, $71.24 millio...",0.000,0.870,0.130,0.8126,0.0574,3063,450,0.000,0.870,0.130,0.8126
2,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0,"$118.71 million, $60.16 million, $46.10 millio...",0.000,0.878,0.122,0.8658,0.0122,2653,602,0.000,0.878,0.122,0.8658
3,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0,Issuance of larger denominated note also econo...,0.014,0.896,0.091,0.8885,0.0075,2425,982,0.014,0.896,0.091,0.8885
4,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,1,The flexibility in detemm ining the income thr...,0.090,0.854,0.057,-0.5948,1.4488,2568,490,0.090,0.854,0.057,-0.5948
5,03 07 2006,"1"" July, 2006\r\n\r\nSBP GOVERNOR MEETS APTMA\...",0,1,"Sympathizing with the textile exporters, Gover...",0.074,0.843,0.083,0.3384,0.5046,3167,991,0.074,0.843,0.083,0.3384
6,27 01 2006,"26"" January, 2006\r\n\r\nSTATE BANK ISSUES ITS...",0,1,The distinguishing feature of the Monetary Pol...,0.000,1.000,0.000,0.0000,1.0000,606,281,0.000,1.000,0.000,0.0000
7,09 06 2006,"9"" June, 2006\r\n\r\nSBP IMPOSES 50% CASH MARG...",0,1,The State Bank has also directed the banks to ...,0.000,0.960,0.040,0.3400,0.6200,1125,355,0.000,0.960,0.040,0.3400
8,18 01 2006,"18"" January, 2006\r\n\r\nPAKISTAN RECEIVES OVE...",0,1,"$99.06 million, $57.66 million, $49.86 million...",0.000,0.920,0.080,0.7430,0.1770,2607,597,0.000,0.920,0.080,0.7430
9,28 06 2006,"27% June, 2006\r\n\r\n \r\n\r\nSTATE BAN K ISS...",0,1,For detailed Guidelines and Prudential Regulat...,0.000,0.839,0.161,0.3182,0.5208,2602,93,0.000,0.839,0.161,0.3182


In [67]:
news_data['neu-compound'] = [news_data['neu'][i] - news_data['compound'][i] for i in range(len(news_data))] #Initialize polarities to zero

In [68]:
news_data.head()

,Date,News,Polarity,Sentiment,Cut,neg,neu,pos,compound,neu-compound,News Length,Cut Length,neg_right,neu_right,pos_right,compound_right
0,01 01 2006,"September 12, 2006\r\n\r\nSBP GOVERNOR, DR. SH...",0,1,"However, she said, policies should be made to ...",0.085,0.841,0.074,-0.2337,1.0747,4056,1082,0.085,0.841,0.074,-0.2337
1,16 06 2006,"16"""" June, 2006\r\n\r\nHISTORIC INCREASE IN WO...",0,0,"$108.71 million, $61.32 million, $71.24 millio...",0.000,0.870,0.130,0.8126,0.0574,3063,450,0.000,0.870,0.130,0.8126
2,17 03 2006,"16'"" March, 2006\r\n\r\nPAKISTAN RECEIVES OVER...",0,0,"$118.71 million, $60.16 million, $46.10 millio...",0.000,0.878,0.122,0.8658,0.0122,2653,602,0.000,0.878,0.122,0.8658
3,26 05 2006,"26"" May, 2006\r\n\r\nState Bank to issue Rs.50...",0,0,Issuance of larger denominated note also econo...,0.014,0.896,0.091,0.8885,0.0075,2425,982,0.014,0.896,0.091,0.8885
4,09 06 2006,"9% June, 2006\r\n\r\nSBP GOVERNOR STRESSES THE...",0,1,The flexibility in detemm ining the income thr...,0.090,0.854,0.057,-0.5948,1.4488,2568,490,0.090,0.854,0.057,-0.5948


In [69]:
news_data.describe()

,Polarity,Sentiment,neg,neu,pos,compound,neu-compound,News Length,Cut Length,neg_right,neu_right,pos_right,compound_right
count,1346.0,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000
mean,0.0,0.142645,0.022783,0.830212,0.097238,0.484605,0.345607,2139.934621,612.303120,0.022783,0.830212,0.097238,0.484605
std,0.0,0.936104,0.035398,0.215221,0.098453,0.478284,0.516145,971.827335,345.575153,0.035398,0.215221,0.098453,0.478284
min,0.0,-1.000000,0.000000,0.000000,0.000000,-0.959800,-0.926400,4.000000,0.000000,0.000000,0.000000,0.000000,-0.959800
25%,0.0,-1.000000,0.000000,0.816000,0.033000,0.000000,-0.062200,1320.500000,321.000000,0.000000,0.816000,0.033000,0.000000
50%,0.0,1.000000,0.000000,0.879000,0.084000,0.648600,0.154450,2279.000000,642.000000,0.000000,0.879000,0.084000,0.648600
75%,0.0,1.000000,0.037000,0.935000,0.138750,0.911575,0.712075,2860.500000,918.000000,0.037000,0.935000,0.138750,0.911575
max,0.0,1.000000,0.285000,1.000000,1.000000,0.994100,1.787100,4419.000000,1282.000000,0.285000,1.000000,1.000000,0.994100


In [70]:
news_data['neu-compound'].quantile([0.33,0.66,0.99])

0.33    0.000000
0.66    0.504600
0.99    1.717015
Name: neu-compound, dtype: float64

In [71]:
for i in range(len(news_data)): 
    if news_data['neu-compound'][i] >= 0.117081:news_data['Sentiment'][i] = 1   
    elif news_data['neu-compound'][i] > 0 and news_data['neu-compound'][i] < 0.117081:news_data['Sentiment'][i] = 0
    if news_data['neu-compound'][i] <= 0:news_data['Sentiment'][i] = -1   

<ipython-input-71-64c110de0910>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if news_data['neu-compound'][i] >= 0.117081:news_data['Sentiment'][i] = 1
<ipython-input-71-64c110de0910>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elif news_data['neu-compound'][i] > 0 and news_data['neu-compound'][i] < 0.117081:news_data['Sentiment'][i] = 0
<ipython-input-71-64c110de0910>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if news_da

In [ ]:
'''for i in range(len(news_data)):
    if news_data['pos'][i] > news_data['neu'][i] and news_data['pos'][i] > news_data['neg'][i]:
        news_data['Sentiment'][i] = 1
    elif news_data['neu'][i] > news_data['pos'][i] and news_data['neu'][i] > news_data['neg'][i]:
        news_data['Sentiment'][i] = 0
    if news_data['neg'][i] > news_data['neu'][i] and news_data['neg'][i] > news_data['pos'][i]:
        news_data['Sentiment'][i] = -1'''

'''for i in range(len(news_data)):
    if news_data['compound'][i] > 0.8: 
        news_data['Sentiment'][i] = 1
    elif news_data['compound'][i] >= 0.5 and news_data['compound'][i] <= 0.8:
        news_data['Sentiment'][i] = 0
    if news_data['compound'][i] < 0.5:  
        news_data['Sentiment'][i] = -1'''  

In [72]:
news_data.describe()

,Polarity,Sentiment,neg,neu,pos,compound,neu-compound,News Length,Cut Length,neg_right,neu_right,pos_right,compound_right
count,1346.0,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000
mean,0.0,0.142645,0.022783,0.830212,0.097238,0.484605,0.345607,2139.934621,612.303120,0.022783,0.830212,0.097238,0.484605
std,0.0,0.936104,0.035398,0.215221,0.098453,0.478284,0.516145,971.827335,345.575153,0.035398,0.215221,0.098453,0.478284
min,0.0,-1.000000,0.000000,0.000000,0.000000,-0.959800,-0.926400,4.000000,0.000000,0.000000,0.000000,0.000000,-0.959800
25%,0.0,-1.000000,0.000000,0.816000,0.033000,0.000000,-0.062200,1320.500000,321.000000,0.000000,0.816000,0.033000,0.000000
50%,0.0,1.000000,0.000000,0.879000,0.084000,0.648600,0.154450,2279.000000,642.000000,0.000000,0.879000,0.084000,0.648600
75%,0.0,1.000000,0.037000,0.935000,0.138750,0.911575,0.712075,2860.500000,918.000000,0.037000,0.935000,0.138750,0.911575
max,0.0,1.000000,0.285000,1.000000,1.000000,0.994100,1.787100,4419.000000,1282.000000,0.285000,1.000000,1.000000,0.994100


### Label data with polarity

In [113]:
count = 0

for i in range(len(news_data)):
    date = check(news_data['Date'][i])
    print(date)
    #print(diff.index())
    
    #index = diff.index[diff['Date'] == date]
    df.index[df['BoolCol']].tolist()
    print(diff[diff['Date']=='date'].index.values)
    #print(diff.index[diff['Date'] == date])   
    #print(diff[index])   
    #print(diff['Open'][index])
    #mean = diff['Open'][index].rolling(30).mean()
    mean = diff['Open'].mean()
    #print(mean[0])
    #std = diff['Open'][index].rolling(30).std()
    std = diff['Open'].std()
    #print(std[0])
    if (index <= len(diff)):
        count+=1
        polarity = pol(float(diff['Open'][index]), mean, std)
        #print(polarity)
        news_data['Polarity'][i] = polarity
    break

2005-12-30 00:00:00
[]


<ipython-input-113-cbd87276df9b>:19: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (index <= len(diff)):


In [76]:
news_data.describe()

,Polarity,Sentiment,neg,neu,pos,compound,neu-compound,News Length,Cut Length,neg_right,neu_right,pos_right,compound_right
count,1346.0,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000
mean,0.0,0.142645,0.022783,0.830212,0.097238,0.484605,0.345607,2139.934621,612.303120,0.022783,0.830212,0.097238,0.484605
std,0.0,0.936104,0.035398,0.215221,0.098453,0.478284,0.516145,971.827335,345.575153,0.035398,0.215221,0.098453,0.478284
min,0.0,-1.000000,0.000000,0.000000,0.000000,-0.959800,-0.926400,4.000000,0.000000,0.000000,0.000000,0.000000,-0.959800
25%,0.0,-1.000000,0.000000,0.816000,0.033000,0.000000,-0.062200,1320.500000,321.000000,0.000000,0.816000,0.033000,0.000000
50%,0.0,1.000000,0.000000,0.879000,0.084000,0.648600,0.154450,2279.000000,642.000000,0.000000,0.879000,0.084000,0.648600
75%,0.0,1.000000,0.037000,0.935000,0.138750,0.911575,0.712075,2860.500000,918.000000,0.037000,0.935000,0.138750,0.911575
max,0.0,1.000000,0.285000,1.000000,1.000000,0.994100,1.787100,4419.000000,1282.000000,0.285000,1.000000,1.000000,0.994100


### Label data with cumulative labels

In [134]:
news_data['Cum'] = [-2 for i in range(len(news_data))] #Initialize polarities to zero

In [135]:
for i in range(len(news_data)):
    
    if news_data['Polarity'][i] == news_data['Sentiment'][i]:
        
        news_data['Cum'][i] = news_data['Polarity'][i]      

<ipython-input-135-2b63794f0991>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_data['Cum'][i] = news_data['Polarity'][i]


In [136]:
news_data = news_data[news_data['Cum'] != -2]
news_data.describe()

,Polarity,Sentiment,neg,neu,pos,compound,neu-compound,Cum
count,140.0,140.0,140.000000,140.000000,140.000000,140.000000,140.000000,140.0
mean,0.0,0.0,0.015450,0.875950,0.108664,0.822240,0.053710,0.0
std,0.0,0.0,0.023346,0.055707,0.045474,0.056639,0.033028,0.0
min,0.0,0.0,0.000000,0.533000,0.063000,0.526700,0.000100,0.0
25%,0.0,0.0,0.000000,0.860750,0.084000,0.802000,0.026425,0.0
50%,0.0,0.0,0.000000,0.887000,0.099000,0.828200,0.056950,0.0
75%,0.0,0.0,0.022750,0.909000,0.121000,0.855500,0.080400,0.0
max,0.0,0.0,0.150000,0.936000,0.423000,0.906200,0.116200,0.0


In [137]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4))
count_pos = news_data['Cum'].value_counts()[1]
count_ntl = news_data['Cum'].value_counts()[0]
count_neg = news_data['Cum'].value_counts()[-1]

plt.bar(['Negative', 'Neutral', 'Positive'], [count_neg, count_ntl, count_pos])
plt.title("Sentiment Data Distribution")

KeyError: 1

<Figure size 576x288 with 0 Axes>

In [ ]:
size = (count_pos + count_neg)//2
temp = news_data.sort_values(by=['Cum'], ascending = False)
neutrals = temp[count_pos:count_ntl]
print(neutrals.head())
import random
filtered = neutrals.sample(n=size)
news_data = news_data[news_data.Polarity != 0]
news_data = pd.concat([news_data, filtered])

news_data.describe()

In [ ]:
plt.figure(figsize=(8,4))
count_pos = news_data['Cum'].value_counts()[1]
count_ntl = news_data['Cum'].value_counts()[0]
count_neg = news_data['Cum'].value_counts()[-1]

plt.bar(['Negative', 'Neutral', 'Positive'], [count_neg, count_ntl, count_pos])
plt.title("Sentiment Data Distribution")

### Save labelled data

In [ ]:
news_data.to_csv(path_or_buf = 'BATA_Good_News' + '.csv', index = False) #Save dataframe as CSV